In [ ]:
!pip install pyspellchecker

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string # Library for string operations

import os

# plotly library
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt #Another plotting libraray

# word cloud library
from wordcloud import WordCloud

#Regex library
import re

#Spell Checker
from spellchecker import SpellChecker 
spell = SpellChecker()

ModuleNotFoundError: No module named 'spellchecker'

In [ ]:
TrainDataSet= pd.read_csv('../input/nlp-getting-started/train.csv')
TestDataSet=pd.read_csv('../input/nlp-getting-started/test.csv')
TrainDataSet.head(3)

In [ ]:
TrainDataSet[TrainDataSet["target"] == 0]["text"].values[0:120]


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=0,stop_words="english", max_features=200)
counts = cv.fit_transform(TrainDataSet["text"][0:500]).toarray().ravel()
words = np.array(cv.get_feature_names()) 
#counts = counts / float(counts.max())
#print(words)
print(counts)

# Data Cleaning

We need to clean the data to avoid errors and incorrect results

* **Remove URL from the tweet**
URL'S some error during processing so we are using regex library to remove the urls

In [ ]:
TrainDataSet['text'] = TrainDataSet['text'].apply(lambda x: re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', str(x)))
TrainDataSet.tail(3)

In [ ]:
TestDataSet['text'] = TestDataSet['text'].apply(lambda x: re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', str(x)))
TestDataSet.tail(3)

* **Remove Emoji from the tweet**
we need to remove emoji from the tweet since people are using lot of emojies in there tweet to express emotions. We need to create function so we can specify different emoji patterns with range of unicode characters, the list is not complete but good for now.

In [ ]:
def EmojiCleanser(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [ ]:
TestDataSet['text'] = TestDataSet['text'].apply(lambda x: EmojiCleanser(str(x)))
TrainDataSet['text'] = TrainDataSet['text'].apply(lambda x: EmojiCleanser(str(x)))
TrainDataSet.tail(3)

* **Remove HTML Tags from Tweet**
we need to remove html tags so we can avoid creating unncessary tokens. we can use a regex expression to remove those tags.

In [ ]:
TestDataSet['text'] = TestDataSet['text'].apply(lambda x: re.sub(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});', '', str(x)))
TrainDataSet['text'] = TrainDataSet['text'].apply(lambda x: re.sub(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});', '', str(x)))
TrainDataSet.tail(3)

* **Remove Punctuations from the Tweet**
we need to remove the puntuations from tweet so we are using string libraray to remove the punctuations

In [ ]:
TestDataSet['text'] = TestDataSet['text'].apply(lambda x: str(x).translate(str.maketrans('','',string.punctuation)))
TrainDataSet['text'] = TrainDataSet['text'].apply(lambda x: str(x).translate(str.maketrans('','',string.punctuation)))
TrainDataSet.tail(3)


*** Spell correction in tweets**
We need to correct the spelling in tweets so we will get more accurate tokens. We can use SpellChecker in pyspellchecker library.

In [ ]:
#TestDataSet['text'] = TestDataSet['text'].apply(lambda x: " ".join([spell.correction(i) for i in str(x).split()]))
#TrainDataSet['text'] = TrainDataSet['text'].apply(lambda x: " ".join([spell.correction(i) for i in str(x).split()]))
#TrainDataSet.tail(3)


In [ ]:
# filling missing values 
#TrainDataSet[['keyword']] = TrainDataSet[['keyword']].fillna('Not Identified')
#TrainDataSet[['location']] = TrainDataSet[['location']].fillna('Not Identified')

TrainDataSet = TrainDataSet[TrainDataSet.keyword.notnull()]
TrainDataSet = TrainDataSet[TrainDataSet.location.notnull()]


Grouped_Disaster = TrainDataSet.groupby(['keyword'])['id'].count().reset_index()
Grouped_Location = TrainDataSet.groupby(['location'])['id'].count().reset_index()

Grouped_Disaster = Grouped_Disaster.query('keyword !="Not Identified"' )
Grouped_Location = Grouped_Location.query('location !="Not Location"' )

Group_Disaster_filter = Grouped_Disaster.sort_values('id',ascending=False)[:20][::-1]
Grouped_Location_filter = Grouped_Location.sort_values('id',ascending=False)[:20][::-1]

fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "bar"}, {"type": "bar"}]],
    subplot_titles=("Top 20 Disaster by Tweets","Top 20 Tweet Location")
)

fig.add_trace(go.Bar(name='id',text='id', x=Group_Disaster_filter['keyword'], y=Group_Disaster_filter['id']),
              row=1, col=1)


fig.add_trace(go.Bar(name='id',text='id', x=Grouped_Location_filter['location'], y=Grouped_Location_filter['id']),
              row=1, col=2)

fig.update_layout(height=700,title_text="Tweets Breakdown", showlegend=False)

fig.show()

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
text = TrainDataSet.text.values
wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'black',
    stopwords = STOPWORDS).generate(str(text))
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
#TrainDataSet.tail(20)
TrainDataSet[TrainDataSet["target"] == 0]["text"].values[0:120]
